## 1.8.1 Stereo retification 
This script intends to address the question: "how many different orientations are needed to have an acceptable 3D error?"


### import  libraries

In [1]:
vscode = 1

In [2]:
if(vscode == 1):
    # for vscode
    %matplotlib qt
else:
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2 
from typing import Sequence
from calib_lib import *
DECIMALS = 2  # how many decimal places to use in print


In [4]:
F = 16                                       # focal length( in mm )
image_size = np.array([11,7])               # sensor size(in mm)
PX= image_size[0]/2.0                       # principal point x-coordinate
PY= image_size[1]/2.0                       # principal point y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                         # yaw angle


# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (13,9)
chess_sq_size = 44

### 1. Create chessboard and translate

In [122]:
translst= []

xrange = np.linspace(-6000,6000,10)
yrange = np.linspace(-4000,4000,10)
zrange = ([20000])
for z in zrange:
    for x in  xrange:
        for y in yrange:
            translst.append([x,y,z])
            
len(translst)

100

In [123]:
# chessboard print
chess_pts = []
world_pts = create_chessboard(chess_dimx,chess_dimy,chess_sq_size)
rotangles = [[0,0,0],[np.pi/4,0,np.pi/2],[0,np.pi/4,np.pi/2],[0,np.pi/4,-np.pi/2],[np.pi/4,0,-np.pi/2]]

for i in range(len(translst)):
    for j in range(len(rotangles)):    
        chess_pts.append(get_chessboard_rot_trans(world_pts,rotangles[j][0],rotangles[j][1],rotangles[j][2],translst[i][0],translst[i][1],translst[i][2]))

chess_pts_arr = np.array(chess_pts)
print(chess_pts_arr.shape)

(500, 140, 3)


### 2. Obtain 2D correspondence
"The technique only requires the camera to observw a planar pattern shown at a few( at least two) different orientations"\
In this example, it is shown 3 different orientations

In [124]:

pattern_size_x,pattern_size_y = (chess_dimx,chess_dimy)

X_pattern = np.linspace(0, pattern_size_x,pattern_size_x + 1)*chess_sq_size
Y_pattern = np.linspace(0, pattern_size_y,pattern_size_y + 1)*chess_sq_size
zdata = np.zeros((pattern_size_x + 1,pattern_size_y + 1))
xdata, ydata = np.meshgrid(X_pattern, Y_pattern)

xdata_ = xdata.flatten()
ydata_ = ydata.flatten()
zdata_ = zdata.flatten()

# homogeneous coordinates
world_pts =([xdata_,ydata_,zdata_])
world_pts_ = np.array(world_pts).T

world_pts_arr = np.zeros((chess_pts_arr.shape[0],3),np.float32)
print(world_pts_arr.shape)
print(world_pts_.shape)
world_pts_arr = world_pts_

world_pts_arr = world_pts_arr.reshape(1,(pattern_size_x+1)*(pattern_size_y+1),3)

x_lst_R = []
x_lst_L = []
for i in range(chess_pts_arr.shape[0]):
    x_arr_R, X_arr_R, E_R, K_R, P_R = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0.0,thetay = 0.0,thetaz = 0.0,trans_x= -C_R[0],trans_y= -C_R[1],trans_z= -C_R[2],F = F)
    x_arr_L, X_arr_L, E_L, K_L, P_L = get_image_points(chess_pts_arr[i,:,:],PX,PY,thetax= 0.0,thetay = 0.0,thetaz = 0.0,trans_x= -C_L[0],trans_y= -C_L[1],trans_z= -C_L[2],F = F)
    
    x_lst_R.append(x_arr_R)
    x_lst_L.append(x_arr_L)

x_zhang_R = np.array(x_lst_R,np.float32).reshape((len(rotangles)*len(translst),(pattern_size_x+1)*(pattern_size_y+1),2))
x_zhang_L = np.array(x_lst_L,np.float32).reshape((len(rotangles)*len(translst),(pattern_size_x+1)*(pattern_size_y+1),2))

world_pts_arr = np.array(world_pts_arr,np.float32).reshape((1,(pattern_size_x+1)*(pattern_size_y+1),3))

print("World points: ",world_pts_arr.shape)
print("Image points: ",x_zhang_L.shape)


(500, 3)
(140, 3)
World points:  (1, 140, 3)
Image points:  (500, 140, 2)


In [125]:
fig = plt.figure(figsize=(20,11))

nr_photo = 0
ax_L = fig.add_subplot(121)
ax_L.plot(x_zhang_L[:,:,0],x_zhang_L[:,:,1], color = 'r',ls = "None", marker = ".", label = 'Left')
ax_L.set_xlabel("X axis")
ax_L.set_ylabel("Y axis")
ax_L.grid()
#ax_L.legend()
ax_L.set_xlim([0,image_size[0]])
ax_L.set_ylim([0,image_size[1]])

ax_R = fig.add_subplot(122)
ax_R.plot(x_zhang_R[:,:,0],x_zhang_R[:,:,1], color = 'r',ls = "None", marker = ".", label = 'Right')
ax_R.set_xlabel("X axis")
ax_R.set_ylabel("Y axis")
ax_R.grid()
#ax_R.legend()
ax_R.set_xlim([0,image_size[0]])
ax_R.set_ylim([0,image_size[1]])
plt.show()

In [104]:
world_pts_arr_ = np.zeros((len(rotangles)*len(translst),world_pts_arr.shape[0],world_pts_arr.shape[1],world_pts_arr.shape[2]))
for i in range(world_pts_arr_.shape[0]):
    world_pts_arr_[i,:,:,:] = world_pts_arr
world_pts_arr_ = np.array(world_pts_arr_,np.float32)

In [105]:
x_zhang_R.shape

(300, 140, 2)

### 3. mono camera calibration

In [106]:
#chess_pts_arr = np.array(chess_pts_arr,np.float32).reshape((3,(pattern_size_x+1)*(pattern_size_y+1),3))
#print(chess_pts_arr.shape)
ret_R, mtx_R, dist_R, rvecs_R, tvecs_R = cv2.calibrateCamera(world_pts_arr_, x_zhang_R, (image_size[0],image_size[1]), None, None)
ret_L, mtx_L, dist_L, rvecs_L, tvecs_L = cv2.calibrateCamera(world_pts_arr_, x_zhang_L, (image_size[0],image_size[1]), None, None)

###  4. Stereo calibration 

In [107]:
stereo_flags = cv2.CALIB_FIX_INTRINSIC

# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(world_pts_arr_, x_zhang_L, x_zhang_R, mtx_L, dist_L, mtx_R, dist_R, (image_size[0],image_size[1]), criteria_stereo, stereo_flags)


In [121]:
ret_L

2.213222840715945e-07

In [108]:
R1,R2,P_L_est,P_R_est,Q,roi_left,roi_right = cv2.stereoRectify(new_mtxL, ret_L, new_mtxR, ret_R, (image_size[0],image_size[1]), Rot, Trns,flags = cv2.CALIB_ZERO_DISPARITY)
print(P_L_est)
print(P_R_est)

[[16.00000127  0.          5.49999988  0.        ]
 [ 0.         16.00000127  3.50000033  0.        ]
 [ 0.          0.          1.          0.        ]]
[[ 1.60000013e+01  0.00000000e+00  5.49999988e+00 -8.00000115e+03]
 [ 0.00000000e+00  1.60000013e+01  3.50000033e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]]


In [109]:
x_R = np.array([7.45678948, 1.00184743])
x_L = np.array([7.79416698, 1.00184743])
XestOpenCV = cv2.triangulatePoints(P_L_est,P_R_est, x_L, x_R) 
Xoriginal = cv2.triangulatePoints(P_L, P_R, x_L, x_R) 


In [110]:
Xoriginal /= Xoriginal[-1]
Xoriginal

array([[ 3.39999997e+03],
       [-3.70231057e+03],
       [ 2.37123104e+04],
       [ 1.00000000e+00]])

In [111]:
XestOpenCV /= XestOpenCV[-1]
XestOpenCV

array([[ 3.40000036e+03],
       [-3.70231130e+03],
       [ 2.37123138e+04],
       [ 1.00000000e+00]])

In [112]:
estError = np.sum(np.abs(XestOpenCV-Xoriginal))
estError

0.0045202544592939375